In [9]:
import numpy as np
import pandas as pd
import os
import datetime as dt
import math
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt

In [10]:
SKIP=False           # false skippa esimerkkidatan latauksen
ORGANIZE=False       # true luo nls dataframen
CREATEFINAL=False    # true luo final dataframen  
CALCULATECORR=False  # true laskee finaldf f statistiikat
PRINT=False          # true tulostaa klusteriplotit




In [11]:

if SKIP:
    path = 'E:/Koulu/data/muu/Alberding_Pasila/pasivrs01-2122-2-checkInit'
    # path = 'E:/Koulu/data/muu/Alberding_joensuu/MJOE-vrs-2275-5-checkInit'
    pasdir = 'E:/Koulu/data/muu/Alberding_Pasila'
    joedir = 'E:/Koulu/data/muu/Alberding_joensuu'

    df = pd.read_csv(path, delimiter=';')
    df['datetime'] = pd.to_datetime(df['Event Time [UTC]']).dt.round('5min')

    # df = df.drop(columns=['startDelay', 'TTFA[s]', 'Sigma HDOP', 
    #                         'Sigma H', 'Sigma NE', 'Epochs', 'Data-Age[s]', 'checkType',
    #                       'Sigma Sat.', 'Sigma Data-Age', 'Sigma N', 'Sigma E', 
    #                       '# of Sat.', 'initTyp', 'Solution', 'Event Time [UTC]'])

    df = df[['datetime', 'dN[cm]', 'dE[cm]', 'dNE[cm]', 'HDOP', 'Sigma NE', 'Sigma N', 'Sigma E']]

    df = df.loc[(df['datetime'].dt.time >= dt.time(6)) & (df['datetime'].dt.time <= dt.time(16))]
    df = df.groupby('datetime').mean().reset_index()
    df

In [12]:
ORGANIZE = False
pasila = []
pasiladates = []
joensuu = []
joensuudates = []
fails = []

dates = pd.read_csv('data/dates.csv')
dates = pd.to_datetime(dates['date']).dt.date.values

if ORGANIZE:    
    i = 1
    nfiles = len(os.listdir(pasdir))

    nlsdf = pd.DataFrame(columns=['datetime', 'dN[cm]', 'dE[cm]', 'dNE[cm]', 'HDOP', 'Sigma NE', 'Sigma N', 'Sigma E', 'asema'])
    
    print('Processing Pasila files...')
    for file in os.listdir(pasdir):
        print(f'Processing file {file}, {i}/{nfiles}, {100*(i/nfiles):.2f}%', end='\r')
        i += 1

        try:
            df = pd.read_csv(os.path.join(pasdir, file), delimiter=';')
            date = pd.to_datetime(df['Event Time [UTC]'][0]).date()
            if date in dates:
                pasila.append(str(file))
                pasiladates.append(date)

                df['datetime'] = pd.to_datetime(df['Event Time [UTC]']).dt.round('5min')

                
                # Define expected columns
                expected_cols = ['datetime', 'dN[cm]', 'dE[cm]', 'dNE[cm]', 'HDOP', 'Sigma NE', 'Sigma N', 'Sigma E']

                # Add missing columns with NaN
                for col in expected_cols:
                    if col not in df.columns:
                        df[col] = pd.NA

                # Select only the expected columns
                df = df[expected_cols]

                
                df = df.loc[(df['datetime'].dt.time >= dt.time(6)) & (df['datetime'].dt.time <= dt.time(16))]
                df = df.groupby('datetime').mean().reset_index()
                df['asema'] = 'pasila'

                nlsdf = pd.concat([nlsdf, df], ignore_index=True)
    
        except:
            fails.append(pasdir+'/'+str(file))
    
    i = 1
    nfiles = len(os.listdir(joedir))
    print('Processing Joensuu files...')
    for file in os.listdir(joedir):
        print(f'Processing file {file}, {i}/{nfiles}, {100*(i/nfiles):.2f}%', end='\r')
        i += 1

        try:
            df = pd.read_csv(os.path.join(joedir, file), delimiter=';')
            date = pd.to_datetime(df['Event Time [UTC]'][0]).date()
            if date in dates:
                joensuu.append(str(file))
                joensuudates.append(date)

                df['datetime'] = pd.to_datetime(df['Event Time [UTC]']).dt.round('5min')
                
                # Define expected columns
                expected_cols = ['datetime', 'dN[cm]', 'dE[cm]', 'dNE[cm]', 'HDOP', 'Sigma NE', 'Sigma N', 'Sigma E']

                # Add missing columns with NaN
                for col in expected_cols:
                    if col not in df.columns:
                        df[col] = pd.NA

                # Select only the expected columns
                df = df[expected_cols]
                
                df = df.loc[(df['datetime'].dt.time >= dt.time(6)) & (df['datetime'].dt.time <= dt.time(16))]
                df = df.groupby('datetime').mean().reset_index()
                df['asema'] = 'joensuu'

                nlsdf = pd.concat([nlsdf, df], ignore_index=True)
    
        except:
            fails.append(joedir+'/'+str(file))
    
    nlsdf = nlsdf.sort_values('datetime')
    nlsdf.to_csv('data/NLSdf.csv', index=False)

    dif1 = set(dates).difference(pasiladates)
    dif2 = set(dates).difference(joensuudates)

    print(sorted(list(dif1&dif2)))
    print()
    print(fails)

else:
    nlsdf = pd.read_csv('data/NLSdf.csv')
    nlsdf['datetime'] = pd.to_datetime(nlsdf['datetime'])
    print(nlsdf.head(10))


             datetime     dN[cm]     dE[cm]    dNE[cm]  HDOP   asema
0 2019-01-16 06:10:00  21.500000  23.500000  31.800000   0.4  pasila
1 2019-01-16 06:30:00  12.000000  28.000000  30.500000   0.4  pasila
2 2019-01-16 06:45:00  33.200000  14.400000  36.200000   0.4  pasila
3 2019-01-16 06:50:00  37.700000   2.600000  37.800000   0.4  pasila
4 2019-01-16 06:55:00  35.000000  -1.225000  35.100000   0.4  pasila
5 2019-01-16 07:00:00  38.033333  -3.566667  38.200000   0.4  pasila
6 2019-01-16 07:05:00  47.566667 -10.400000  48.733333   0.4  pasila
7 2019-01-16 07:10:00  62.625000 -12.850000  63.975000   0.4  pasila
8 2019-01-16 07:15:00  64.533333 -17.066667  66.766667   0.4  pasila
9 2019-01-16 07:20:00  70.233333  -7.033333  70.633333   0.4  pasila


In [13]:

if CREATEFINAL:

    hairiopv = nlsdf.loc[(nlsdf['datetime'].dt.date == dt.date(2023,2,27))]
    clusters = pd.read_csv('data/clusterdf.csv', sep=',')
    clusters['dates'] = pd.to_datetime(clusters['dates'])

    finaldf = pd.merge(nlsdf, clusters, how='left', left_on=nlsdf['datetime'].dt.date, right_on=clusters['dates'].dt.date)
    finaldf = finaldf.drop(columns=['key_0', 'dates'])
    finaldf[list(clusters.columns[1:])] = finaldf[list(clusters.columns[1:])].astype('category')
    finaldf
else:
    finaldf = pd.read_csv('data/finaldf.csv', sep=';')
    finaldf['datetime'] = pd.to_datetime(finaldf['datetime'])
    finaldf['date'] = pd.to_datetime(finaldf['date'])
    print(finaldf.head(10))

             datetime     dN[cm]     dE[cm]    dNE[cm]  HDOP  Sigma NE  \
0 2019-01-16 06:10:00  21.500000  23.500000  31.800000   0.4       NaN   
1 2019-01-16 06:30:00  12.000000  28.000000  30.500000   0.4       NaN   
2 2019-01-16 06:45:00  33.200000  14.400000  36.200000   0.4       NaN   
3 2019-01-16 06:50:00  37.700000   2.600000  37.800000   0.4       NaN   
4 2019-01-16 06:55:00  35.000000  -1.225000  35.100000   0.4       NaN   
5 2019-01-16 07:00:00  38.033333  -3.566667  38.200000   0.4       NaN   
6 2019-01-16 07:05:00  47.566667 -10.400000  48.733333   0.4       NaN   
7 2019-01-16 07:10:00  62.625000 -12.850000  63.975000   0.4       NaN   
8 2019-01-16 07:15:00  64.533333 -17.066667  66.766667   0.4       NaN   
9 2019-01-16 07:20:00  70.233333  -7.033333  70.633333   0.4       NaN   

   Sigma N  Sigma E   asema  k2  ...  k8  k9  k10  k15  k25  k30       date  \
0      NaN      NaN  pasila   1  ...   6   1    1    2    1   23 2019-01-16   
1      NaN      NaN  pasila

In [14]:
nlsdf.corr(numeric_only=True)

,dN[cm],dE[cm],dNE[cm],HDOP
dN[cm],1.000000,0.220673,0.859692,-0.056711
dE[cm],0.220673,1.000000,0.184712,-0.045678
dNE[cm],0.859692,0.184712,1.000000,-0.025928
HDOP,-0.056711,-0.045678,-0.025928,1.000000


In [15]:
if CALCULATECORR:

    from scipy.stats import f_oneway

    for col in finaldf.columns:
        if col.startswith('k'):
            groups = [group['HDOP'].dropna().values for _, group in finaldf.groupby(col)]
            if len(groups) > 1:
                stat, p = f_oneway(*groups)
                print(f"{col}: F-statistic = {stat:.2f}, p-value = {p:.4e}")

In [16]:
if CALCULATECORR:    
    from statsmodels.stats.multicomp import pairwise_tukeyhsd

    tukey = pairwise_tukeyhsd(endog=finaldf['HDOP'], groups=finaldf['k3'], alpha=0.05)
    print(tukey.summary())

In [17]:
data_dir = "data/keog/"

os.environ['OMP_NUM_THREADS'] = '1'
dataframes = []
dates = []

for file in os.listdir(data_dir):
    if file.endswith('.csv'):
        df = pd.read_csv(os.path.join(data_dir, file))
        dataframes.append(df)

heatmaps = []  
heatmaps_imputed = []  

imputer = SimpleImputer(strategy='mean')

for df in dataframes:
    XX, YY = np.meshgrid(df['time_seconds'].unique(), df['gdlat'].unique())
    ZZ = df['blrmvd'].values
    ZZ = np.reshape(ZZ, XX.T.shape).T

    heatmaps.append(ZZ)

    hmi = imputer.fit_transform(ZZ).flatten()
    if hmi.size == 1125:
        heatmaps_imputed.append(hmi)
        first_date = df['datetime'].iloc[0][:10]
        dates.append(first_date)
    else:
        continue

heatmaps_imputed = np.vstack(heatmaps_imputed)

c:\Users\mariav\AppData\Local\anaconda3\envs\gradu\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: [106]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
c:\Users\mariav\AppData\Local\anaconda3\envs\gradu\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: [59 60 61 62]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


In [18]:
k = 6


if PRINT:
    for i in range(k):
        labels = np.array(clusters[f'k{k}'])
        cluster_indices = np.where(labels == i)[0]
        n_images = len(cluster_indices)
        print(f"Cluster {i}: {n_images} heatmaps")

        # Setup subplot grid
        n_cols = 5
        n_rows = math.ceil(n_images / n_cols)
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(3*n_cols, 2*n_rows))
        axes = axes.flatten()  # Flatten for easy indexing

        for j, idx in enumerate(cluster_indices):
            ax = axes[j]
            ax.imshow(heatmaps[idx], aspect='auto', cmap='viridis')
            ax.set_title(f"{dates[idx]}")
            ax.axis('off')  # Hide axes for cleaner layout

        # Hide unused subplots
        for b in range(j+1, len(axes)):
            axes[b].axis('off')

        fig.suptitle(f'Cluster {i}', fontsize=16)
        fig.tight_layout(rect=[0, 0, 1, 0.96])  # Leave space for title
        plt.show()